# Testing Individual components of the FV HE scheme

In [1]:
from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encrypter import Encrypter
from syft.frameworks.torch.he.fv.decrypter import Decrypter
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/ravikantsingh/Desktop/PySyft/venv/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'



## Generates coeff_modulus using Polymodulus degree and bit-length parameters.
Generates random integer of the given bit length for using as coeff_modulus

In [2]:
params = EncryptionParams()
cm = CoeffModulus()
params.coeff_modulus = cm.create(4096, [30, 50, 60])
print("coeff_modulus: ", params.coeff_modulus)

coeff_modulus:  [1073692673, 1125899906826241, 1152921504606830593]


## Use standard values of coeff_modulus for the given Poly Modulus degree and security_level.
Need some refractoring and integration of code.

In [3]:
params_standard = EncryptionParams()
cm_standard = CoeffModulus()
for i in SeqLevelType:
    for j in [1024, 4096, 32768]:           # ... [1024, 2048, 4096, 8192, 16384, 32768]
        params_standard.coeff_modulus = cm_standard.bfv_default(j, i)
        print(f"coeff_modulus for {i} and poly_modulus = {j}", params_standard.coeff_modulus)
    print('\n')

coeff_modulus for SeqLevelType.TC128 and poly_modulus = 1024 [132120577]
coeff_modulus for SeqLevelType.TC128 and poly_modulus = 4096 [68719403009, 68719230977, 137438822401]
coeff_modulus for SeqLevelType.TC128 and poly_modulus = 32768 [36028797017456641, 36028797014704129, 36028797014573057, 36028797014376449, 36028797013327873, 36028797013000193, 36028797012606977, 36028797010444289, 36028797009985537, 36028797005856769, 36028797005529089, 36028797005135873, 36028797003694081, 36028797003563009, 36028797001138177, 72057594037338113]


coeff_modulus for SeqLevelType.TC192 and poly_modulus = 1024 [520193]
coeff_modulus for SeqLevelType.TC192 and poly_modulus = 4096 [33538049, 33349633, 33292289]
coeff_modulus for SeqLevelType.TC192 and poly_modulus = 32768 [18014398506729473, 18014398505943041, 18014398499848193, 18014398498799617, 18014398498275329, 36028797017456641, 36028797014704129, 36028797014573057, 36028797014376449, 36028797013327873, 36028797013000193]


coeff_modulus for Se

## Keygeneration

In [4]:
params = EncryptionParams(128, CoeffModulus().create(128, [40, 40]), 1<<6 )
ctx = Context(params)
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [5]:
print('secret key values : ', sk.data)
print(sk)

secret key values :  tensor([            0,             1,             0, 1099511607040,
                    1, 1099511607040,             0, 1099511607040,
        1099511607040,             0,             0, 1099511607040,
                    1,             1,             0, 1099511607040,
                    0,             1,             1, 1099511607040,
        1099511607040,             1, 1099511607040,             0,
                    1,             1, 1099511607040,             1,
        1099511607040,             1,             1,             0,
        1099511607040,             0,             0, 1099511607040,
                    0,             1,             0,             0,
                    0,             0,             1,             1,
        1099511607040, 1099511607040, 1099511607040,             0,
        1099511607040,             0,             0,             0,
        1099511607040, 1099511607040,             1,             1,
        1099511607040,     

In [13]:
print('public key values : ', pk.data)
print(pk)

public key values :  [[tensor(0), tensor(926338339671), tensor(1), tensor(343623747282), tensor(111998154084), tensor(21541215752), tensor(0), tensor(923538336383), tensor(4470200389), tensor(0), tensor(0), tensor(713086186495), tensor(330482760182), tensor(390105111580), tensor(1), tensor(1039656338495), tensor(1), tensor(639107084176), tensor(1077112679102), tensor(768853219080), tensor(146133806542), tensor(610999334908), tensor(429509144121), tensor(1), tensor(1069267655976), tensor(1096476462334), tensor(734562530218), tensor(313613802683), tensor(769148218926), tensor(551663375977), tensor(1048045545820), tensor(1), tensor(451393525139), tensor(3), tensor(1099511607040), tensor(708692148279), tensor(1), tensor(344200810267), tensor(2), tensor(1099511607036), tensor(0), tensor(1099511607040), tensor(292676095009), tensor(10764874096), tensor(545792873552), tensor(69009178180), tensor(124833529244), tensor(4), tensor(1005381688773), tensor(0), tensor(0), tensor(1), tensor(378248691

## Integer Encoder
Encodes Integer values to Plaintext object

In [7]:
int_encoder = IntegerEncoder(ctx)
plaintext = int_encoder.encode(0x12345678)
print(plaintext)
print('plaintext data',plaintext.data)

plaintext data [0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1]


### Decodes back to Integer

In [8]:
print(int_encoder.decode(plaintext))

305419896


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [9]:
encrypter = Encrypter(ctx, pk)
ciphertext = encrypter.encrypt(plaintext)
print(ciphertext)
print('ciphertext data :', ciphertext.data)

ciphertext data : [[tensor(223338295178), tensor(1039018689007), tensor(4), tensor(1), tensor(738697101791), tensor(21541215751), tensor(1), tensor(923538336382), tensor(1099511607040), tensor(1099511607040), tensor(1099511607040), tensor(0), tensor(330482760181), tensor(0), tensor(1), tensor(0), tensor(1), tensor(639107084176), tensor(1), tensor(768853219080), tensor(111607748856), tensor(128603406854), tensor(751455665262), tensor(1099511607036), tensor(527857486749), tensor(295922453706), tensor(0), tensor(313613802685), tensor(390544490621), tensor(1099511607040), tensor(0), tensor(1), tensor(1099511607039), tensor(1), tensor(1099511607040), tensor(751336365566), tensor(5), tensor(3), tensor(1099511607040), tensor(1), tensor(4), tensor(5), tensor(322010528108), tensor(10764874098), tensor(545792873559), tensor(1099511607038), tensor(124833529244), tensor(1099511607040), tensor(390441847626), tensor(0), tensor(1), tensor(1), tensor(1099511607040), tensor(21609406713), tensor(5196348

Encrypt Plaintext to ciphertext using secret_key

In [10]:
encrypter_symmetric = Encrypter(ctx, sk)
ciphertext_symmetric = encrypter_symmetric.encrypt(plaintext)
print(ciphertext_symmetric.data)

[[tensor(223338295174), tensor(980472977070), tensor(2), tensor(717388761444), tensor(69253868208), tensor(326780750251), tensor(1099511607039), tensor(876513201903), tensor(979382042120), tensor(1), tensor(7), tensor(481193864120), tensor(77267322556), tensor(882771645584), tensor(0), tensor(77567442311), tensor(0), tensor(223024296723), tensor(1090788365599), tensor(700228820577), tensor(515736565916), tensor(139627584162), tensor(262013307312), tensor(1099511607040), tensor(804608544324), tensor(85689612406), tensor(116310563817), tensor(732173152214), tensor(386834992754), tensor(398687535810), tensor(1050360288377), tensor(1099511607039), tensor(958129113639), tensor(1099511607037), tensor(2), tensor(326802376877), tensor(0), tensor(1076020874582), tensor(0), tensor(1), tensor(1099511607040), tensor(0), tensor(298847048878), tensor(883492538673), tensor(738895992308), tensor(1005138318639), tensor(605776597071), tensor(1099511607039), tensor(704708506222), tensor(2), tensor(109951

## Decrypter
Decrypt Ciphertext to Plaintext using secret_key

In [11]:
decrypter = Decrypter(ctx, sk)
result = decrypter.decrypt(ciphertext)
print("len : ",len(result.data))
print(result.data)
print(result)

len :  2
[tensor(31), tensor(34)]


In [12]:
print(int_encoder.decode(result))

tensor(-29)
